In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

2020-12-16 10:12:01,482 - NumExpr defaulting to 8 threads.


In [2]:
path = '../raw_data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2344518937184) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2344518936032) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2344518936896) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2344518938048) was called 1 times
Running: Function "load_module_lookup" (id=2344518935312) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [10]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Blue']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel, fault_agg_level= 'None', agg_type = 'count')

In [14]:
av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = True, availability_select_options = 'None')

av_agg = feat.aggregate_availability(av_sel, agg_level = 'None')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'None')

AttributeError: 'str' object has no attribute 'keys'

In [15]:
av_agg.columns

NameError: name 'av_agg' is not defined

In [13]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, target = 'Blue Tote Loss' ,agg_level = None,remove_0=True)

NameError: name 'av_agg' is not defined

In [ ]:
X,y = md.gen_feat_var(df)
y=1-y

In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test)

maxf = None

# X_sel_rf = md.select_features(X, y, RF_mdl, thres = -np.inf, max_feat = maxf)

X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options={'thres': 'mean', 'max_feat': None})

X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)


cv_R2 = md.cross_validate_r2(Linear_mdl, X_sel_lm, y, n_folds = 10, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(RF_mdl, X_sel_rf, y, n_folds = 10, shuffle = True, random_state = 101)

cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y, n_folds = 10, shuffle = True, random_state = 101)

#md.stats_model(Linear_mdl,X_test,y_test)

### Add LM - metrics

In [ ]:
sc.lm_coefficients(Linear_mdl, X, y)

In [ ]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

In [ ]:
# plt.figure(figsize=(20,5))
# sns.barplot(data=Coeff, x='Feature',y = 'Coefficients')

In [ ]:
plt.scatter(predictions_LM,y_test)